In [1]:
import os
import glob
import torch
import json
import numpy as np
from tqdm import tqdm
import ray.cloudpickle as pickle
from Helper.ml_models import *  # Enthält z. B. TrainedModel und K_Fold_Dataset


/home/jan/anaconda3/envs/studi/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2025-03-21 15:35:24.423218: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-21 15:35:25.436242: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
BASE_PATH = "/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation"

# Checkpoints und Ausgabeordner
CHECKPOINT_DIR = os.path.join(BASE_PATH, "FINAL_DATEN/hyper_city")
CONF_MATRIX_DIR = os.path.join(BASE_PATH, "FINAL_DATEN/confusion_matrices_city")
os.makedirs(CONF_MATRIX_DIR, exist_ok=True)

# Cityscapes: 20 Klassen
NUM_CLASSES = 20


k_fold_dataset = K_Fold_Dataset(
    image_dir='CityscapesDaten/images',
    annotation_dir='CityscapesDaten/semantic',
    k_fold_csv_dir='Daten/CityscapesDaten',
    leave_out_fold=0
)
k_fold_dataset.check_for_data_leaks()
print(f"Cityscapes Test-Dataset: {len(k_fold_dataset.test_dataset)} Samples gefunden.")


No data leaks found.
Cityscapes Test-Dataset: 695 Samples gefunden.


In [4]:
def load_checkpointed_model_city(model_name, checkpoint_path, num_classes):
    """
    Lädt ein Cityscapes-Modell aus dem Checkpoint.
    Nutzt die Klasse TrainedModel aus Helper.ml_models.
    """
    model = TrainedModel(
        model_name=model_name,
        width=2048,      # Passen Sie ggf. die Eingabegröße an
        height=1024,     # Passen Sie ggf. die Eingabegröße an
        weights_name='',
        skip_local_load=True,
        num_classes=num_classes
    )
    with open(checkpoint_path, "rb") as fp:
        checkpoint_data = pickle.load(fp)
    model.model.load_state_dict(checkpoint_data["model_state"], strict=True)
    if "optimizer_state" in checkpoint_data:
        model.optimizer.load_state_dict(checkpoint_data["optimizer_state"])
    model.model.eval()
    return model

def compute_confusion_matrix(predicted, ground_truth, num_classes):
    """
    Berechnet die Confusion Matrix für ein einzelnes Bild.
    """
    mask = (ground_truth >= 0) & (ground_truth < num_classes)
    label = num_classes * ground_truth[mask] + predicted[mask]
    count = torch.bincount(label, minlength=num_classes**2)
    confusion_matrix = count.reshape(num_classes, num_classes)
    return confusion_matrix

In [5]:
# Liste aller Checkpoint-Dateien (.pkl) im CHECKPOINT_DIR
checkpoint_files = glob.glob(os.path.join(CHECKPOINT_DIR, "*.pkl"))
print(f"Found {len(checkpoint_files)} checkpoint(s) in {CHECKPOINT_DIR}")

for checkpoint_path in checkpoint_files:
    # Extrahiere den Modellnamen aus dem Dateinamen, z.B.:
    # "deeplabv3_mobilenet_v3_large_best_checkpoint.pkl" -> "deeplabv3_mobilenet_v3_large"
    base_name = os.path.basename(checkpoint_path)
    model_name = base_name.split("_best_checkpoint.pkl")[0]
    print(f"\nProcessing model: {model_name}")
    print(f"Using checkpoint: {checkpoint_path}")
    
    try:
        model_loaded = load_checkpointed_model_city(model_name, checkpoint_path, NUM_CLASSES)
    except Exception as e:
        print(f"[ERROR] Could not load model {model_name}: {e}")
        continue
    
    # Initialisiere eine leere Confusion Matrix
    confusion_matrix_total = torch.zeros((NUM_CLASSES, NUM_CLASSES), dtype=torch.int64)
    
    # Evaluierung über den Test-Datensatz
    with torch.no_grad():
        for i in tqdm(range(len(k_fold_dataset.test_dataset)), desc=f"Evaluating {model_name}"):
            image, annotation = k_fold_dataset.test_dataset[i]
            # Führe Inferenz durch – hier wird angenommen, dass model_loaded.inference(image) den Logits-Tensor liefert
            output = model_loaded.inference(image)
            predicted = output.argmax(1).squeeze(0)
            
            cm = compute_confusion_matrix(predicted.cpu(), annotation.cpu(), NUM_CLASSES)
            confusion_matrix_total += cm
    
    # Speichere die resultierende Confusion Matrix
    save_path = os.path.join(CONF_MATRIX_DIR, f"{model_name}_confusion_matrix.pt")
    torch.save(confusion_matrix_total, save_path)
    print(f"Confusion Matrix for {model_name} saved to: {save_path}")


Found 6 checkpoint(s) in /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/hyper_city

Processing model: fcn_resnet101
Using checkpoint: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/hyper_city/fcn_resnet101_best_checkpoint.pkl
Using CUDA GPU
Model loaded: fcn_resnet101 | Device: cuda 
Skipping local .pth load logic (likely using external Ray checkpoint).


Evaluating fcn_resnet101: 100%|██████████| 695/695 [01:12<00:00,  9.64it/s]


Confusion Matrix for fcn_resnet101 saved to: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/confusion_matrices_city/fcn_resnet101_confusion_matrix.pt

Processing model: deeplabv3_mobilenet_v3_large
Using checkpoint: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/hyper_city/deeplabv3_mobilenet_v3_large_best_checkpoint.pkl
Using CUDA GPU
Model loaded: deeplabv3_mobilenet_v3_large | Device: cuda 
Skipping local .pth load logic (likely using external Ray checkpoint).


Evaluating deeplabv3_mobilenet_v3_large: 100%|██████████| 695/695 [00:51<00:00, 13.50it/s]


Confusion Matrix for deeplabv3_mobilenet_v3_large saved to: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/confusion_matrices_city/deeplabv3_mobilenet_v3_large_confusion_matrix.pt

Processing model: fcn_resnet50
Using checkpoint: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/hyper_city/fcn_resnet50_best_checkpoint.pkl
Using CUDA GPU
Model loaded: fcn_resnet50 | Device: cuda 
Skipping local .pth load logic (likely using external Ray checkpoint).


...

Confusion Matrix for lraspp_mobilenet_v3_large saved to: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/confusion_matrices_city/lraspp_mobilenet_v3_large_confusion_matrix.pt

Processing model: deeplabv3_resnet101
Using checkpoint: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/hyper_city/deeplabv3_resnet101_best_checkpoint.pkl
Using CUDA GPU
Model loaded: deeplabv3_resnet101 | Device: cuda 
Skipping local .pth load logic (likely using external Ray checkpoint).


Evaluating deeplabv3_resnet101: 100%|██████████| 695/695 [01:07<00:00, 10.28it/s]


Confusion Matrix for deeplabv3_resnet101 saved to: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/confusion_matrices_city/deeplabv3_resnet101_confusion_matrix.pt

Processing model: deeplabv3_resnet50
Using checkpoint: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/hyper_city/deeplabv3_resnet50_best_checkpoint.pkl
Using CUDA GPU
Model loaded: deeplabv3_resnet50 | Device: cuda 
Skipping local .pth load logic (likely using external Ray checkpoint).


Evaluating deeplabv3_resnet50: 100%|██████████| 695/695 [01:01<00:00, 11.37it/s]

Confusion Matrix for deeplabv3_resnet50 saved to: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/confusion_matrices_city/deeplabv3_resnet50_confusion_matrix.pt
